In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [4]:
#count number of schools
school_count = school_data_complete['school_name'].nunique()

#total number of students
stu_count = school_data_complete['Student ID'].nunique()

# Total budget
total_budget = sum(school_data_complete['budget'].unique())

# Average reading score 
avg_reading_score = school_data_complete['reading_score'].mean()

# Average math score
avg_math_score = school_data_complete['math_score'].mean()


# Percentage of students with a score >70%
pass_math_pct = (school_data_complete[school_data_complete['math_score'] >= 70].math_score.count()) / stu_count

# Percentage of students with reading score of >70%
pass_reading_pct = (school_data_complete[school_data_complete['reading_score'] >= 70].reading_score.count()) / stu_count

# average score of students who passed both math and reading
overall_passing_rate = (pass_math_pct + pass_reading_pct)/2

# Store all calculations into dataframe
district_summary = pd.DataFrame({"Total Schools":[school_count],
                                 "Total Students":[stu_count],
                                 "Total Budget":[total_budget],
                                 "Average Math Score":[avg_math_score],
                                 "Average Reading Score":[avg_reading_score],
                                 "% Passing Math":[pass_math_pct*100],
                                 "% Passing Reading":[pass_reading_pct*100],
                                 "% Overall Passing Rate":[overall_passing_rate*100]})

# Reformat Total Students and Total Budget
district_summary['Total Students'] = district_summary.apply(lambda x: "{:,.0f}".format(x['Total Students']), axis=1)
district_summary['Total Budget'] = district_summary.apply(lambda x: "${:,.2f}".format(x['Total Budget']), axis=1)

# Print Summary Dataframe, round the numbers to two decimal places
summary = district_summary.round(2)
summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.39


In [6]:
#school summary, make dataframe with school names as index
school_summary = school_data_complete.groupby(['school_name'])

#get a list of school names 
school_names = school_data_complete['school_name'].sort_values().unique()

#get a list of school types
school_types = school_data.sort_values(by="school_name").type

#count the total amount of students in each school
school_total_students = list(school_summary['student_name'].count())

#produce a list of each schools budget
school_budget = list(school_summary['budget'].mean())

#take the budget and divide by number of students in the school to geta per student budget
school_per_student_budget = [i/j for i,j in zip(school_budget,school_total_students)]

#make a list of each schools average math and reading score

school_avg_math_score = list(school_summary.math_score.mean())
school_avg_reading_score = list(school_summary.reading_score.mean())

# Calculating passing percentages per school

school_summary = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])
school_pct_passing_math = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),school_total_students)]
school_summary = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name'])
school_pct_passing_reading = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),school_total_students)]
school_overall_passing = [(i+j)/2 for i,j in zip(school_pct_passing_math,school_pct_passing_reading)]

# Compile all calculations into dataframe
school_summary_df = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":school_total_students,
                                  "Total School Budget":school_budget,
                                  "Per Student Budget":school_per_student_budget,
                                  "Average Math Score":school_avg_math_score,
                                  "Average Reading Score":school_avg_reading_score,
                                  "% Passing Math":school_pct_passing_math,
                                  "% Passing Reading":school_pct_passing_reading,
                                  "Overall Passing Rate":school_overall_passing})

#Drop the original index and round the floating numbers 
school_summary_df = school_summary_df.reset_index(drop=True)
school_summary_df = school_summary_df.round(2)
school_summary_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928.0,628.0,77.05,81.03,66.68,81.93,74.31
1,Cabrera High School,Charter,1858,1081356.0,582.0,83.06,83.98,94.13,97.04,95.59
2,Figueroa High School,District,2949,1884411.0,639.0,76.71,81.16,65.99,80.74,73.36
3,Ford High School,District,2739,1763916.0,644.0,77.10,80.75,68.31,79.30,73.80
4,Griffin High School,Charter,1468,917500.0,625.0,83.35,83.82,93.39,97.14,95.27
5,Hernandez High School,District,4635,3022020.0,652.0,77.29,80.93,66.75,80.86,73.81
6,Holden High School,Charter,427,248087.0,581.0,83.80,83.81,92.51,96.25,94.38
7,Huang High School,District,2917,1910635.0,655.0,76.63,81.18,65.68,81.32,73.50
8,Johnson High School,District,4761,3094650.0,650.0,77.07,80.97,66.06,81.22,73.64
9,Pena High School,Charter,962,585858.0,609.0,83.84,84.04,94.59,95.95,95.27


In [7]:
#find the top 5 schools by using the sort_values()
top_schools = school_summary_df.sort_values(by='Overall Passing Rate', ascending=False).head(5).reset_index(drop=True)
top_schools

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,1858,1081356.0,582.0,83.06,83.98,94.13,97.04,95.59
1,Thomas High School,Charter,1635,1043130.0,638.0,83.42,83.85,93.27,97.31,95.29
2,Griffin High School,Charter,1468,917500.0,625.0,83.35,83.82,93.39,97.14,95.27
3,Pena High School,Charter,962,585858.0,609.0,83.84,84.04,94.59,95.95,95.27
4,Wilson High School,Charter,2283,1319574.0,578.0,83.27,83.99,93.87,96.54,95.20


In [8]:
#Repeat the same thing as above but search for lowest 5 schools
bottom_five = school_summary_df.sort_values(by='Overall Passing Rate', ascending=True).head(5).reset_index(drop=True)
bottom_five

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,3999,2547363.0,637.0,76.84,80.74,66.37,80.22,73.29
1,Figueroa High School,District,2949,1884411.0,639.0,76.71,81.16,65.99,80.74,73.36
2,Huang High School,District,2917,1910635.0,655.0,76.63,81.18,65.68,81.32,73.50
3,Johnson High School,District,4761,3094650.0,650.0,77.07,80.97,66.06,81.22,73.64
4,Ford High School,District,2739,1763916.0,644.0,77.10,80.75,68.31,79.30,73.80


In [22]:
#look through the original dataframe and group by school name through each grade. Then take the mean() 
ninth = school_data_complete.loc[school_data_complete['grade']=='9th'].groupby("school_name")
ninth_m = ninth['math_score'].mean()
tenth = school_data_complete.loc[school_data_complete['grade']=='10th'].groupby("school_name")
tenth_m = tenth['math_score'].mean()
eleventh = school_data_complete.loc[school_data_complete['grade']=='11th'].groupby("school_name")
eleventh_m = eleventh['math_score'].mean()
twelfth = school_data_complete.loc[school_data_complete['grade']=='12th'].groupby("school_name")
twelfth_m = twelfth['math_score'].mean()

# Create dataframe for math scores summary
math_summary = pd.DataFrame({"9th": ninth_m,
                            "10th": tenth_m,
                            "11th": eleventh_m,
                            "12th": twelfth_m})

# Display mean math scores per school per grade

math_summary.round(2)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [26]:
# Do the same thing as above just for reading instead
ninth_r = school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby("school_name")
avg_ninth_r = ninth_r['reading_score'].mean()
tenth_r = school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby("school_name")
avg_tenth_r = tenth_r['reading_score'].mean()
eleventh_r = school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby("school_name")
avg_eleventh_r = eleventh_r['reading_score'].mean()
twelfth_r = school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby("school_name")
avg_twelfth_r = twelfth_r['reading_score'].mean()

# Create dataframe for reading scores summary 
read_summary = pd.DataFrame({"9th": avg_ninth_r,
                            "10th": avg_tenth_r,
                            "11th": avg_eleventh_r,
                            "12th": avg_twelfth_r})

read_summary.round(2)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [41]:
#Create bins, I used example bins 
bins = [0,585,615,645,675]

# Create names for the bins
spending_range = ['<$585','$585-615','$615-645','$645-675']


school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], 
                                                         bins, labels=spending_range)
spend_summary = school_summary_df.groupby("Spending Ranges (Per Student)")
spend_summary = spend_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                               "% Passing Reading", "Overall Passing Rate"]]
spend_summary.mean().round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46,96.61,95.04
$585-615,83.60,83.88,94.23,95.90,95.06
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


In [39]:
School_size_bins = [0, 1000, 2000, 5000]

# Create names for bins
s_m_l = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], 
                                                         School_size_bins, labels=s_m_l)
size_summary = school_summary_df.groupby("School Size")
size_summary = size_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                             "% Passing Reading", "Overall Passing Rate"]]
size_summary.mean().round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.87,93.60,96.79,95.20
Large (2000-5000),77.74,81.34,69.96,82.77,76.36


In [36]:
#show the difference in average math and reading scores between district and charter 
type_summary = school_summary_df.groupby("School Type")
type_summary = type_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                             "% Passing Reading", "Overall Passing Rate"]]
type_summary.mean().round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
